In [1]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import folium
import numpy as np
from branca.colormap import StepColormap

ox.settings.use_cache = True

# 1) Vilnius city polygon
vilnius = ox.geocode_to_gdf("Vilnius, Lithuania").to_crs(4326)

# 2) Get ALL admin boundaries that touch Vilnius
raw_admin = ox.features_from_polygon(
    vilnius.geometry.iloc[0],
    tags={"boundary": "administrative"}
).to_crs(4326)

# 3) Normalise admin_level and keep ONLY level 10 (urban elderships)
raw_admin["admin_level"] = raw_admin["admin_level"].astype(str).str.strip()
elderships10 = raw_admin[
    (raw_admin["admin_level"] == "10")
    & (raw_admin.geometry.type.isin(["Polygon", "MultiPolygon"]))
].copy()

# Prefer Lithuanian names if present
if "name:lt" in elderships10.columns:
    elderships10["name"] = elderships10["name"].fillna(elderships10["name:lt"])

# Keep only proper elderships (exclude sub-elderships if any slipped in)
elderships10 = elderships10[elderships10["name"].str.contains("seniūnija", na=False)].copy()

# Now it's safe to trim columns
elderships = elderships10[["name", "geometry"]]
print(len(elderships), "eldership polygons")
print(sorted(elderships["name"].tolist()))


21 eldership polygons
['Antakalnio seniūnija', 'Fabijoniškių seniūnija', 'Grigiškių seniūnija', 'Justiniškių seniūnija', 'Karoliniškių seniūnija', 'Lazdynų seniūnija', 'Naujamiesčio seniūnija', 'Naujininkų seniūnija', 'Naujosios Vilnios seniūnija', 'Panerių seniūnija', 'Pašilaičių seniūnija', 'Pilaitės seniūnija', 'Rasų seniūnija', 'Senamiesčio seniūnija', 'Verkių seniūnija', 'Vilkpėdės seniūnija', 'Viršuliškių seniūnija', 'Šeškinės seniūnija', 'Šnipiškių seniūnija', 'Žirmūnų seniūnija', 'Žvėryno seniūnija']


In [2]:
# Listings with lat/lon already present
df = pd.read_csv("aruodas_rent_enriched_20November.csv")
df_pts = df.dropna(subset=["latitude", "longitude", "price_per_m2"]).copy()

gdf_pts = gpd.GeoDataFrame(
    df_pts,
    geometry=gpd.points_from_xy(df_pts["longitude"], df_pts["latitude"]),
    crs="EPSG:4326"
)

joined = gpd.sjoin(gdf_pts, elderships, how="inner", predicate="within")
med_ppm2 = (joined.groupby("name", as_index=False)["price_per_m2"]
                  .median().rename(columns={"price_per_m2": "med_price_per_m2"}))
elderships_avg = elderships.merge(med_ppm2, on="name", how="left")
elderships_avg

,name,geometry,med_price_per_m2
0,Senamiesčio seniūnija,"POLYGON ((25.273 54.681, 25.273 54.681, 25.273...",19.100
1,Naujamiesčio seniūnija,"POLYGON ((25.247 54.679, 25.247 54.679, 25.247...",17.840
2,Žvėryno seniūnija,"POLYGON ((25.237 54.692, 25.233 54.694, 25.234...",17.670
3,Vilkpėdės seniūnija,"POLYGON ((25.179 54.657, 25.18 54.657, 25.182 ...",14.330
4,Panerių seniūnija,"POLYGON ((25.027 54.624, 25.04 54.63, 25.04 54...",14.205
5,Grigiškių seniūnija,"POLYGON ((25.071 54.69, 25.071 54.69, 25.071 5...",11.480
6,Lazdynų seniūnija,"POLYGON ((25.178 54.673, 25.19 54.681, 25.192 ...",13.760
7,Karoliniškių seniūnija,"POLYGON ((25.194 54.682, 25.195 54.684, 25.196...",13.045
8,Pilaitės seniūnija,"POLYGON ((25.105 54.701, 25.103 54.702, 25.101...",14.000
9,Justiniškių seniūnija,"POLYGON ((25.199 54.713, 25.199 54.713, 25.199...",12.500


In [3]:
# --- 1) Build 5 quantile bins (robust to outliers) ---
vals = elderships_avg["med_price_per_m2"].dropna().to_numpy()
quantiles = np.quantile(vals, [0, .2, .4, .6, .8, 1.0])

# Ensure strictly increasing (fallback if data are flat)
if len(np.unique(quantiles)) < 6:
    quantiles = np.linspace(vals.min(), vals.max(), 6)

# --- 2) High-contrast green → yellow → red palette (cheap→expensive) ---
palette = ["#2DC937", "#99C140", "#E7B416", "#DB7B2B", "#CC3232"]

cmap = StepColormap(
    colors=palette,
    vmin=float(quantiles[0]),
    vmax=float(quantiles[-1]),
    index=quantiles.tolist(),
    caption="Median rent (€/m²)"
)

# --- 3) Map (keep your Street/Satellite baselayers & bounds as before) ---
m = folium.Map(
    location=[54.6872, 25.2797],
    zoom_start=12, tiles=None,
    min_zoom=11, max_zoom=16
)

folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}",
    attr="© Esri", name="Street", no_wrap=True
).add_to(m)
folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="© Esri", name="Satellite", no_wrap=True
).add_to(m)

def style_fn(feat):
    v = feat["properties"].get("med_price_per_m2")
    return {
        "fillColor": "#d9d9d9" if v is None else cmap(v),
        "fillOpacity": 0.82,
        "color": "#374151",  # darker border for contrast
        "weight": 1.2
    }

folium.GeoJson(
    elderships_avg,
    style_function=style_fn,
    tooltip=folium.GeoJsonTooltip(
        fields=["name", "med_price_per_m2"],
        aliases=["Eldership", "Median €/m²"],
        localize=True,
        labels=True,
        sticky=False
    ),
    control=False
).add_to(m)

# Legend + tight bounds
cmap.add_to(m)
minx, miny, maxx, maxy = elderships_avg.total_bounds
pad = 0.02
bounds = [[miny - pad, minx - pad], [maxy + pad, maxx + pad]]
m.fit_bounds(bounds)
map_id = m.get_name()
m.get_root().html.add_child(folium.Element(f"""
<script>
  var bounds = L.latLngBounds([{bounds[0][0]}, {bounds[0][1]}], [{bounds[1][0]}, {bounds[1][1]}]);
  {map_id}.setMaxBounds(bounds);
  {map_id}.options.maxBoundsViscosity = 1.0;
</script>
"""))

folium.LayerControl(collapsed=False, position="topright").add_to(m)
m.save("vilnius_rent_ppm2_by_eldership.html")
